In [3]:
import json
from collections import defaultdict

In [4]:
data_path = "fine_tuning_dataset.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 25
First example:
{'role': 'system', 'content': "You are an assistant geared towards helping users overcome writer's block when writing journal entries. When given the text from a user's past journal entry, you are to respond with questions to help the user with writing their next journal entry."}
{'role': 'user', 'content': "Here are previous journal entries from a user:\nToday was a day filled with unexpected twists and turns, yet it turned out to be quite fulfilling in its own right. The morning began with a sense of anticipation, as I had planned to tackle a long list of tasks, both professional and personal. As I sipped my morning coffee, I took a moment to review my schedule and set my intentions for the day. Little did I know that fate had other plans in store. The first surprise of the day came in the form of a phone call from an old friend whom I hadn't spoken to in years. Hearing their voice filled me with nostalgia and joy, as we reminisced about shared memorie

In [5]:
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found
